# Example: CitiBike data

Adapted from Kelsey Jordahl
https://gist.github.com/kjordahl/5957573

In [3]:
import requests  
import json
from pyproj import Proj
from shapely.geometry import Point
import geopandas as gp

NYC borough boundaries downloaded from [Bytes of the Big Apple](http://www.nyc.gov/html/dcp/download/bytes/nybb_13a.zip)

In [4]:
boros = gp.GeoDataFrame.from_file('nybb_15b/nybb.shp')

Load real time bike station data from [CitiBike](http://citibikenyc.com) json API:

In [5]:
# get the bike data and convert it to a ditcionary
endpoint_url = 'http://citibikenyc.com/stations/json'
response = requests.get(endpoint_url)
data = json.loads(response.text)


In [6]:
# convert the relevant part to a geodataframe
df = gp.GeoDataFrame(data['stationBeanList'])
df.head()

,altitude,availableBikes,availableDocks,city,id,landMark,lastCommunicationTime,latitude,location,longitude,postalCode,stAddress1,stAddress2,stationName,statusKey,statusValue,testStation,totalDocks
0,,21,12,,304,,2019-02-19 03:48:41 PM,40.704633,,-74.013617,,Broadway & Battery Pl,,Broadway & Battery Pl,1,In Service,False,33
1,,56,3,,359,,2019-02-19 03:47:55 PM,40.755103,,-73.974987,,E 47 St & Park Ave,,E 47 St & Park Ave,1,In Service,False,64
2,,8,36,,377,,2019-02-19 03:45:30 PM,40.722438,,-74.005664,,6 Ave & Canal St,,6 Ave & Canal St,1,In Service,False,45
3,,3,31,,402,,2019-02-19 03:48:26 PM,40.740343,,-73.989551,,Broadway & E 22 St,,Broadway & E 22 St,1,In Service,False,39
4,,33,8,,520,,2019-02-19 03:46:34 PM,40.759923,,-73.976485,,W 52 St & 5 Ave,,W 52 St & 5 Ave,1,In Service,False,41


In [7]:
# there is one row for each bike station.  How many stations are there? 
len(df)

813

In [8]:
# in the file above, there are lon-lats, but no geometry field
# we need to set that up

s = gp.GeoSeries([Point(x, y) for x, y in zip(df['longitude'], df['latitude'])])
df['geometry'] = s
df.crs = {'init': 'epsg:4326', 'no_defs': True}
df.geometry.total_bounds

array([-74.0836394 ,  40.65539977, -73.9077436 ,  40.81439444])

In [14]:
# make sure they are on the same CRS.  
# checking the bounds is a nice way of seeing this
df.to_crs(boros.crs, inplace=True)
df.geometry.total_bounds

array([ 961064.73659426,  178056.37414829, 1009803.53178432,
        235986.42166716])

In [11]:
# the geometry objects can do lots of cool stuff.  For example: 

manhattan = boros.geometry[3]
in_mn = df.geometry.within(manhattan)
print(sum(in_mn), 'stations in Manhattan')

413 stations in Manhattan


# Your turn

You can read about the range of operations available in geopandas here: 
    
http://geopandas.org/index.html

Your assignment is to: 

1. Calculate how many stations are in each borough
2. Calculate how many bikes are currently available in each borough
3. Read about and try at least two new spatial or geometric operations (beyond what I've covered here). 

In [9]:
import requests  
import json
from pyproj import Proj
from shapely.geometry import Point
import geopandas as gp


In [11]:
boros = gp.GeoDataFrame.from_file('nybb_15b/nybb.shp')
endpoint_url = 'http://citibikenyc.com/stations/json'
response = requests.get(endpoint_url)
data = json.loads(response.text)
len(df)

813

In [33]:
bikes= citibikes_boros.groupby('BoroName')['availableBikes'].sum()
bikes

NameError: name 'citibikes_boros' is not defined

In [35]:
manhattan = boros.geometry[3]
in_mn = df.geometry.within(manhattan)
print(sum(in_mn), 'availableBikes')

0 availableBikes


In [16]:
print (sum(df['availableBikes'][in_mn]), 'available bikes in Manhattan')

0 available bikes in Manhattan


In [23]:
brooklyn = boros.geometry[2]
in_bk = df.geometry.within(brooklyn)
print (sum(df['availableBikes'][in_bk]), 'available bikes in Brooklyn')

0 available bikes in Brooklyn


In [29]:
s=boros.BoroName.to_frame()
for index, row in boros.iterrows():
    s[row.BoroName]=boros.geometry.touches(row.geometry)
s

,BoroName,Staten Island,Brooklyn,Queens,Manhattan,Bronx
0,Staten Island,False,False,False,False,False
1,Brooklyn,False,False,True,True,False
2,Queens,False,True,False,True,True
3,Manhattan,False,True,True,False,True
4,Bronx,False,False,True,True,False
